In [1]:
from pathlib import Path
import sys
import time

import numpy as np
import pandas as pd
import torch as T
import torch.nn.functional as F
import torch.utils.data as tdata

sys.path.append('..')
from agent.config import load_config, override_config
from agent.state.model import Supervisor
from agent.state.data import Dataset

CONFIG_FILE = Path('../train_conf.yaml')
DATA_DIR = Path('../data')

In [2]:
conf = load_config(CONFIG_FILE)
conf = override_config(conf, skip_failed=True, state={'decay_base': 0})
dataset = Dataset(DATA_DIR, conf)
targets = []
for t in dataset.iter_targets():
    t /= conf.state.target_scale
    sum_ = np.sum(t, axis=0)
    row = np.concatenate([t.shape[:1], sum_])
    targets.append(row)

In [3]:
cols = ('steps',) + Dataset.TARGET_COLS
df = pd.DataFrame(targets, columns=cols)
df.describe()

,steps,coal,cobblestone,crafting_table,dirt,furnace,iron_axe,iron_ingot,iron_ore,iron_pickaxe,...,planks,stick,stone,stone_axe,stone_pickaxe,torch,wooden_axe,wooden_pickaxe,diamond,end
count,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,...,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.000000,67.0,67.0
mean,16606.358209,21.850746,182.029851,7.432836,8.343284,4.059701,0.014925,9.000000,20.492537,2.955224,...,21.805970,20.492537,33.059701,0.134328,3.850746,21.343284,0.194030,2.447761,1.0,1.0
std,11936.070116,21.868930,137.522667,3.568481,8.282058,3.209270,0.122169,8.651502,17.049281,2.920320,...,13.658429,10.468384,27.729691,0.574602,3.192101,30.077452,0.679545,1.917097,0.0,0.0
min,4139.000000,0.000000,23.000000,4.000000,0.000000,2.000000,0.000000,2.000000,5.000000,1.000000,...,8.000000,6.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.0,1.0
25%,8564.000000,10.500000,71.500000,5.000000,3.500000,2.000000,0.000000,4.000000,11.000000,1.000000,...,13.000000,12.500000,13.500000,0.000000,2.000000,0.000000,0.000000,1.000000,1.0,1.0
50%,12225.000000,17.000000,152.000000,6.000000,6.000000,3.000000,0.000000,6.000000,15.000000,1.000000,...,18.000000,18.000000,28.000000,0.000000,3.000000,13.000000,0.000000,2.000000,1.0,1.0
75%,23501.500000,26.500000,237.000000,9.000000,11.000000,5.000000,0.000000,13.500000,25.000000,4.000000,...,26.500000,25.000000,42.000000,0.000000,4.500000,27.500000,0.000000,3.000000,1.0,1.0
max,65989.000000,140.000000,572.000000,23.000000,40.000000,22.000000,1.000000,51.000000,98.000000,14.000000,...,92.000000,54.000000,149.000000,4.000000,15.000000,168.000000,4.000000,10.000000,1.0,1.0


In [4]:
df.median().sort_values(ascending=False)

steps             12225.0
cobblestone         152.0
stone                28.0
planks               18.0
stick                18.0
coal                 17.0
iron_ore             15.0
log                  14.0
torch                13.0
iron_ingot            6.0
dirt                  6.0
crafting_table        6.0
stone_pickaxe         3.0
furnace               3.0
wooden_pickaxe        2.0
diamond               1.0
iron_pickaxe          1.0
end                   1.0
stone_axe             0.0
iron_axe              0.0
wooden_axe            0.0
dtype: float64